# いろんな準備

In [ ]:
# WandBを準備
# インストール
!pip install wandb -qU
import wandb
from google.colab import userdata
wandb_api_key = userdata.get('WANDB_API_KEY')
# Login
!wandb login $wandb_api_key

In [ ]:
# Gドライブをマウント
from google.colab import drive
drive.mount('/content/drive')

# ディレクトリを準備
!mkdir -p data
drive_path = "/content/drive/MyDrive/Colab Notebooks/DLBasics2023_colab/final-vqa"
data_path = f"{drive_path}/data"

In [ ]:
# 学習データの読み込み
!rsync -ah --no-i-r --info=progress2 "{data_path}/train.json" "./data/"
!rsync -ah --no-i-r --info=progress2 "{data_path}/valid.json" "./data/"
!rsync -ah --no-i-r --info=progress2 "{data_path}/train.zip" "./data/"
!rsync -ah --no-i-r --info=progress2 "{data_path}/valid.zip" "./data/"

In [ ]:
# 学習データを解凍
!apt install -y -qq parallel > /dev/null

%cd /content/data
!zipinfo -1 train.zip | parallel --no-notice --bar "unzip -qn train.zip {}"
!zipinfo -1 valid.zip | parallel --no-notice --bar "unzip -qn valid.zip {}"

## ここからがベースライン


In [ ]:
import re
import random
import time
import datetime as dt

from statistics import mode

from PIL import Image
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torchvision
from torchvision import transforms

In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
# 工夫：高速化
torch.backends.cudnn.benchmark = True

In [ ]:
# 質問文の前処理１（工夫：うまく行かなかった）
from nltk.stem import PorterStemmer
import nltk
nltk.download('wordnet')

def stemming(question):
    stemmer = PorterStemmer()
    return ' '.join([stemmer.stem(word) for word in question.split()])

In [ ]:
# 質問文の前処理２（工夫：OK）
from nltk.corpus import stopwords
nltk.download('stopwords')

def remove_stopwords(question):
    stop_words = set(stopwords.words('english'))
    return ' '.join([word for word in question.split() if word not in stop_words])

In [ ]:
def process_text(text):
    # lowercase
    text = text.lower()

    # 数詞を数字に変換
    num_word_to_digit = {
        'zero': '0', 'one': '1', 'two': '2', 'three': '3', 'four': '4',
        'five': '5', 'six': '6', 'seven': '7', 'eight': '8', 'nine': '9',
        'ten': '10'
    }
    for word, digit in num_word_to_digit.items():
        text = text.replace(word, digit)

    # 小数点のピリオドを削除
    text = re.sub(r'(?<!\d)\.(?!\d)', '', text)

    # 冠詞の削除
    text = re.sub(r'\b(a|an|the)\b', '', text)

    # 短縮形のカンマの追加
    contractions = {
        "dont": "don't", "isnt": "isn't", "arent": "aren't", "wont": "won't",
        "cant": "can't", "wouldnt": "wouldn't", "couldnt": "couldn't"
    }
    for contraction, correct in contractions.items():
        text = text.replace(contraction, correct)

    # 句読点をスペースに変換
    text = re.sub(r"[^\w\s':]", ' ', text)

    # 句読点をスペースに変換
    text = re.sub(r'\s+,', ',', text)

    # 連続するスペースを1つに変換
    text = re.sub(r'\s+', ' ', text).strip()


    # 質問文の前処理：工夫
    text = remove_stopwords(text)
    # text = stemming(text)  # うまく動かなかった

    return text

In [ ]:
# データローダーの作成
class VQADataset(torch.utils.data.Dataset):
    def __init__(self, df_path, image_dir, transform=None, answer=True):
        self.transform = transform  # 画像の前処理
        self.image_dir = image_dir  # 画像ファイルのディレクトリ
        self.df = pd.read_json(df_path)  # 画像ファイルのパス，question, answerを持つDataFrame
        self.answer = answer

        # question / answerの辞書を作成
        self.question2idx = {}
        self.answer2idx = {}
        self.idx2question = {}
        self.idx2answer = {}

        # 質問文に含まれる単語を辞書に追加
        for question in self.df["question"]:
            question = process_text(question)
            words = question.split(" ")
            for word in words:
                if word not in self.question2idx:
                    self.question2idx[word] = len(self.question2idx)
        self.idx2question = {v: k for k, v in self.question2idx.items()}  # 逆変換用の辞書(question)

        if self.answer:
            # 回答に含まれる単語を辞書に追加
            for answers in self.df["answers"]:
                for answer in answers:
                    word = answer["answer"]
                    word = process_text(word)
                    if word not in self.answer2idx:
                        self.answer2idx[word] = len(self.answer2idx)
            self.idx2answer = {v: k for k, v in self.answer2idx.items()}  # 逆変換用の辞書(answer)

    def update_dict(self, dataset):
        """
        検証用データ，テストデータの辞書を訓練データの辞書に更新する．

        Parameters
        ----------
        dataset : Dataset
            訓練データのDataset
        """
        self.question2idx = dataset.question2idx
        self.answer2idx = dataset.answer2idx
        self.idx2question = dataset.idx2question
        self.idx2answer = dataset.idx2answer

    def __getitem__(self, idx):
        """
        対応するidxのデータ（画像，質問，回答）を取得．

        Parameters
        ----------
        idx : int
            取得するデータのインデックス

        Returns
        -------
        image : torch.Tensor  (C, H, W)
            画像データ
        question : torch.Tensor  (vocab_size)
            質問文をone-hot表現に変換したもの
        answers : torch.Tensor  (n_answer)
            10人の回答者の回答のid
        mode_answer_idx : torch.Tensor  (1)
            10人の回答者の回答の中で最頻値の回答のid
        """
        image = Image.open(f"{self.image_dir}/{self.df['image'][idx]}")
        image = self.transform(image)
        question = np.zeros(len(self.idx2question) + 1)  # 未知語用の要素を追加
        question_words = self.df["question"][idx].split(" ")
        for word in question_words:
            try:
                question[self.question2idx[word]] = 1  # one-hot表現に変換
            except KeyError:
                question[-1] = 1  # 未知語


        if self.answer:
            answers = [self.answer2idx[process_text(answer["answer"])] for answer in self.df["answers"][idx]]
            mode_answer_idx = mode(answers)  # 最頻値を取得（正解ラベル）

            return image, torch.Tensor(question), torch.Tensor(answers), int(mode_answer_idx)

        else:
            return image, torch.Tensor(question)

    def __len__(self):
        return len(self.df)


In [ ]:
# 2. 評価指標の実装
# 簡単にするならBCEを利用する
def VQA_criterion(batch_pred: torch.Tensor, batch_answers: torch.Tensor):
    total_acc = 0.

    for pred, answers in zip(batch_pred, batch_answers):
        acc = 0.
        for i in range(len(answers)):
            num_match = 0
            for j in range(len(answers)):
                if i == j:
                    continue
                if pred == answers[j]:
                    num_match += 1
            acc += min(num_match / 3, 1)
        total_acc += acc / 10

    return total_acc / len(batch_pred)

In [ ]:
# 3. モデルの実装
# ResNetを利用できるようにしておく
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channels: int, out_channels: int, stride: int = 1):
        super().__init__()

        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        residual = x
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))

        out += self.shortcut(residual)
        out = self.relu(out)

        return out

In [ ]:
class BottleneckBlock(nn.Module):
    expansion = 4

    def __init__(self, in_channels: int, out_channels: int, stride: int = 1):
        super().__init__()

        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(out_channels, out_channels * self.expansion, kernel_size=1, stride=1)
        self.bn3 = nn.BatchNorm2d(out_channels * self.expansion)
        self.relu = nn.ReLU(inplace=True)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels * self.expansion:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels * self.expansion, kernel_size=1, stride=stride),
                nn.BatchNorm2d(out_channels * self.expansion)
            )

    def forward(self, x):
        residual = x
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))

        out += self.shortcut(residual)
        out = self.relu(out)

        return out

In [ ]:
class ResNet(nn.Module):
    def __init__(self, block, layers):
        super().__init__()
        self.in_channels = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(block, layers[0], 64)
        self.layer2 = self._make_layer(block, layers[1], 128, stride=2)
        self.layer3 = self._make_layer(block, layers[2], 256, stride=2)
        self.layer4 = self._make_layer(block, layers[3], 512, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, 512)

    def _make_layer(self, block, blocks, out_channels, stride=1):
        layers = []
        layers.append(block(self.in_channels, out_channels, stride))
        self.in_channels = out_channels * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, out_channels))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x


def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])


def ResNet50():
    return ResNet(BottleneckBlock, [3, 4, 6, 3])

In [ ]:
class VQAModel(nn.Module):
    def __init__(self, vocab_size: int, n_answer: int):
        super().__init__()
        self.resnet = ResNet18()
        self.text_encoder = nn.Linear(vocab_size, 512)

        self.fc = nn.Sequential(
            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, n_answer)
        )

    def forward(self, image, question):
        image_feature = self.resnet(image)              # 画像の特徴量
        question_feature = self.text_encoder(question)  # テキストの特徴量

        x = torch.cat([image_feature, question_feature], dim=1)
        x = self.fc(x)

        return x

In [ ]:
# 4. 学習の実装
def train(model, dataloader, optimizer, criterion, device):
    model = model.to(device, non_blocking=True)
    model.train()

    total_loss = 0
    total_acc = 0
    simple_acc = 0

    start = time.time()
    for image, question, answers, mode_answer in dataloader:
        image = image.to(device, non_blocking=True)
        question = question.to(device, non_blocking=True)
        answers = answers.to(device, non_blocking=True)
        mode_answer = mode_answer.to(device, non_blocking=True)

        pred = model(image, question)
        loss = criterion(pred, mode_answer.squeeze())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_acc += VQA_criterion(pred.argmax(1), answers)    # VQA accuracy
        simple_acc += (pred.argmax(1) == mode_answer).float().mean().item()  # simple accuracy

    return total_loss / len(dataloader), total_acc / len(dataloader), simple_acc / len(dataloader), time.time() - start

In [ ]:
# 評価関数
def eval(model, dataloader, optimizer, criterion, device):
    model.eval()

    total_loss = 0
    total_acc = 0
    simple_acc = 0

    start = time.time()
    with torch.no_grad():
      for image, question, answers, mode_answer in dataloader:
          image = image.to(device, non_blocking=True)
          question = question.to(device, non_blocking=True)
          answers = answers.to(device, non_blocking=True)
          mode_answer = mode_answer.to(device, non_blocking=True)

          pred = model(image, question)
          loss = criterion(pred, mode_answer.squeeze())

          total_loss += loss.item()
          total_acc += VQA_criterion(pred.argmax(1), answers)   # VQA accuracy
          simple_acc += (pred.argmax(1) == mode_answer).float().mean().item()  # simple accuracy

    return total_loss / len(dataloader), total_acc / len(dataloader), simple_acc / len(dataloader), time.time() - start


# 便利なツール

In [ ]:
# WandB Tool
def start_wandb():
    wandb.init()
    return  wandb.config


def epoch_log(epoch, train_loss, train_acc, train_simple_acc, train_time, valid_loss, valid_acc, valid_simple_acc, valid_time):
    # wandbへ出力
    wandb.log({
      "train_time": train_time,
      "train_loss": train_loss,
      "train_acc": train_acc,
      "train_simple-acc": train_simple_acc,
      "valid_time": valid_time,
      "valid_loss": valid_loss,
      "valid_acc": valid_acc,
      "valid_simple_acc-acc": valid_simple_acc,
      "epoch": epoch
    })

    # セルへ出力
    print(f"【{epoch}】\n"
      f"t-time: {train_time:.2f} [s]\n"
      f"t-loss: {train_loss:.4f}\n"
      f"t-acc: {train_acc:.4f}\n"
      f"t-simple-acc: {train_simple_acc:.4f}\n"
      f"v-time: {valid_time:.2f} [s]\n"
      f"v-loss: {valid_loss:.4f}\n"
      f"v-acc: {valid_acc:.4f}\n"
      f"v-simple-acc: {valid_simple_acc:.4f}\n"
    )

In [ ]:
# Early Stopping クラス
class EarlyStopping:
    def __init__(self, mode='loss', patience=10, delta=0, path='checkpoint.pth'):
        self.mode = mode
        self.patience = patience
        self.delta = delta
        self.path = path

        self.early_stop = False

        self.counter = 0
        self.val_loss_min = float('inf')
        self.val_acc_max = 0
        self.best_epoch = None

    def __call__(self, epoch_no, val_loss, val_acc, model, optimizer):
        # Update By Minimun Loss
        if self.mode == 'loss' :
            if val_loss <= self.val_loss_min :
                self.val_loss_min = val_loss
                self.val_acc_max = val_acc
                self.best_epoch = epoch_no
                self.save_checkpoint(epoch_no, val_loss, val_acc, model, optimizer)

                self.counter = 0

            else: # case: val_loss > self.val_loss_min
                if val_loss > self.val_loss_min + self.delta :
                    self.counter += 1
                    if self.counter >= self.patience:
                        self.early_stop = True
                else:
                    pass

        # Update By Max Accuracy
        else: # self.mode == "acc"
            if val_acc >= self.val_acc_max :
                self.val_acc_max = val_acc
                self.val_loss_min = val_loss
                self.best_epoch = epoch_no
                self.save_checkpoint(epoch_no, val_loss, val_acc, model, optimizer)

                self.counter = 0

            else: # case: val_acc < self.val_acc_max
                if val_acc < self.val_acc_max - self.delta :
                    self.counter += 1
                    if self.counter >= self.patience:
                        self.early_stop = True
                else:
                    pass

        return self.early_stop

    def save_checkpoint(self, epoch_no, val_loss, val_acc, model, optimizer):
        '''Saves model when validation loss decrease.'''
        torch.save({
            'epoch_no': epoch_no,
            'val_loss': val_loss,
            'val_acc': val_acc,
            'model_state_dict': model.cpu().state_dict(),   # [Attension!] Move model to CPU
            'optimizer_state_dict': optimizer.state_dict(),
        }, self.path)

# 学習開始

In [ ]:
# deviceの設定
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# EarlyStop Parameters
earlystop_patience = 5
earlystop_threshold = 0.1
tmp_best_model_filename = "temporary_best_model.pth"

def build_earlystopping():
    # EarlyStoppingのインスタンス
    early_stopping = EarlyStopping(
                        mode='acc',
                        patience=earlystop_patience,
                        delta=earlystop_threshold,
                        path=tmp_best_model_filename
                      )

    return early_stopping

# データを準備


In [ ]:
def build_datasets(split_ratio = 0.8):

    # ----------------------
    # 画像変換ロジックtransformを設定
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])

    # ----------------------
    # データセットの取り出し（学習用）
    train_val_dataset = VQADataset(df_path="./data/train.json", image_dir="./data/train", transform=transform)

    # データセットの長さを取得
    train_val_dataset_size = len(train_val_dataset)

    # トレーニングセットとテストセットのサイズ
    train_size = int(split_ratio * train_val_dataset_size)
    valid_size = train_val_dataset_size - train_size

    # トレーニングセットとテストセットに分割
    train_dataset, valid_dataset = torch.utils.data.random_split(train_val_dataset, [train_size, valid_size])

    # 辞書をサブセットにコピー
    train_dataset.question2idx = train_val_dataset.question2idx
    train_dataset.answer2idx = train_val_dataset.answer2idx
    train_dataset.idx2question = train_val_dataset.idx2question
    train_dataset.idx2answer = train_val_dataset.idx2answer

    # 辞書をサブセットにコピー
    valid_dataset.question2idx = train_val_dataset.question2idx
    valid_dataset.answer2idx = train_val_dataset.answer2idx
    valid_dataset.idx2question = train_val_dataset.idx2question
    valid_dataset.idx2answer = train_val_dataset.idx2answer

    # ----------------------
    # データセットの取り出し（提出用）
    test_dataset = VQADataset(df_path="./data/valid.json", image_dir="./data/valid", transform=transform, answer=False)
    test_dataset.update_dict(train_val_dataset)

    return train_dataset, valid_dataset, test_dataset



In [ ]:
# データローダーの作成
def get_dataloader(train_dataset, valid_dataset, test_dataset, batch_size, num_workers=4):
    """
      --- 高速化フラグ  num_workers=4, pin_memory=True, pin_memory_device=device ---
    """
    # 学習・検証用のデータローダ
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True, pin_memory_device=device)
    valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True, pin_memory_device=device)

    # 提出結果作成用のデータローダ
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=num_workers, pin_memory=True, pin_memory_device=device)

    return train_loader, valid_loader, test_loader



# モデルを準備

In [ ]:
# model
def build_model(vocab_size, n_answer):
    model = VQAModel( vocab_size=vocab_size, n_answer = n_answer).to(device, non_blocking=True)
    return model

---
---
# 学習

In [ ]:
# wandbの設定(Sweepsの実行)
sweep_config = {
    'method': 'random',
    'metric': {
        'name': 'valid_acc',
        'goal': 'maximize'
    },
    'parameters': {
        'epochs': {
            # 'values': [1, 3, 5]
            'values': [5]
        },
        'batch_size': {
            # 'values': [32, 64, 128]
            'values': [128]
        },
        'lr': {
            # 'values': [1e-2, 1e-4, 1e-5, 1e-6]
            'values': [1e-4]
        }
    }
}

In [ ]:
# グローバル変数の準備
model = None
optimizer = None
criterion = None

train_dataset = None
valid_dataset= None
test_dataset = None

train_loader = None
valid_loader = None
test_loader = None

In [ ]:
# 学習関数(for Sweep)

def main():

    global model, optimizer, criterion, train_dataset, valid_dataset, test_dataset, train_loader, valid_loader, test_loader

    # --------------------------
    wandb.init()
    config = wandb.config
    early_stopping = build_earlystopping()

    # Dataset and Dataloader
    train_dataset, valid_dataset, test_dataset = build_datasets()
    train_loader, valid_loader, test_loader = get_dataloader(train_dataset, valid_dataset, test_dataset, config.batch_size)

    # Model and optimizer
    model = build_model(vocab_size=len(train_dataset.question2idx)+1, n_answer=len(train_dataset.answer2idx))
    optimizer = torch.optim.Adam(model.parameters(), lr=config.lr)
    criterion = nn.CrossEntropyLoss()

    # --------------------------
    # train　loop
    for epoch in range(1, config.epochs+1):
        # Trainning and Varidatiion
        train_loss, train_acc, train_simple_acc, train_time = train(model, train_loader, optimizer, criterion, device)
        valid_loss, valid_acc, valid_simple_acc, valid_time = eval(model, valid_loader, optimizer, criterion, device)

        # ログ
        epoch_log(epoch, train_loss, train_acc, train_simple_acc, train_time, valid_loss, valid_acc, valid_simple_acc, valid_time)

        # EarlyStop処理
        if early_stopping(epoch, valid_loss, valid_acc, model, optimizer) :
          print(f"======>>>> Early Stopped!!!:   best_valid_acc:{early_stopping.val_acc_max :4f}")
          break


In [ ]:
# 乱数を固定
set_seed(42)
# Sweep
sweep_id = wandb.sweep(sweep_config, project='vqa-baseline-sweep')

##### 学習実行！ ####
wandb.agent(sweep_id, function=main, count=1)


---
---
# 結果出力

In [ ]:
# === ファイル名の生成 ===
from google.colab import files
import datetime
import pytz

# タイムスタンプ文字列を作成
utc_now = datetime.datetime.now()
jst_now = utc_now.astimezone(pytz.timezone('Asia/Tokyo'))
formatted_date = jst_now.strftime("%Y-%m-%dT%H%M%S")

In [ ]:
%%time
# 最終エポック
npy_filename = f"VQA_submission_pred-{formatted_date}-last.npy"
print(npy_filename)

# 提出用ファイル
model = model.to(device, non_blocking=True)
model.eval()
submission = []
for image, question in test_loader:
    image = image.to(device, non_blocking=True)
    question = question.to(device, non_blocking=True)
    print(f"question: {question}")

    pred = model(image, question)
    pred = pred.argmax(1).cpu().item()
    print(f"pred: {pred}")


    submission.append(pred)

submission = [train_dataset.idx2answer[id] for id in submission]
submission = np.array(submission)

# npyファイルをダウンロード
np.save(npy_filename, submission)
files.download(npy_filename)

---
---
# Best結果を出力

In [ ]:
# BESTモデルの再ロード
checkpoint = torch.load(tmp_best_model_filename, map_location=torch.device('cpu'))

model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
val_loss = checkpoint['val_loss']
val_acc = checkpoint['val_acc']
best_epoch_id = checkpoint['epoch_no']

print(f"best_epoch_id: {best_epoch_id}, val_loss: {val_loss:.4f}, val_acc: {val_acc:.4f}\n")

In [ ]:
# BESTエポックの結果を出力
npy_filename = f"VQA_submission_pred-{formatted_date}-best.npy"
print(npy_filename)

# 提出用ファイルの作成
model = model.to(device, non_blocking=True)

model.eval()
submission = []

with torch.no_grad():
  for image, question in test_loader:
      image = image.to(device, non_blocking=True)
      question = question.to(device, non_blocking=True)

      pred = model(image, question)
      pred = pred.argmax(1).cpu().item()
      submission.append(pred)

submission = [train_dataset.idx2answer[id] for id in submission]
submission = np.array(submission)

# ローカルにnpyファイルをダウンロードする
np.save(npy_filename, submission)
files.download(npy_filename)